In [7]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from graphviz import Digraph

def load_data(file_path: str, split: bool = True):
    """
    Load data from file_path and return numpy data

    Parameters
    ----------
    file_path : str
        The path of input data file (tab separated).
    split : bool
        Whether or not to return test set.

    Returns
    ----------
    (X_train, y_train) if split = False 
    (X_train, y_train), (X_test, y_test) if split = True
    """
    try:
        data = pd.read_csv(file_path, sep='\t')
        print(f"Đã đọc file {file_path}: {data.shape} (hàng, cột)")

        if data.shape[1] < 2:
            raise ValueError("File dữ liệu phải có ít nhất 2 cột")

        # Lấy X và y từ dữ liệu
        X = data.iloc[:, :-1]
        y = data.iloc[:, -1]
        
        # Chuyển y thành số nguyên nếu là chuỗi
        y_encoder = LabelEncoder()
        if y.dtype == object:
            y = y_encoder.fit_transform(y)
            print(f"Đã mã hóa nhãn y: {np.unique(y)}")

        # Nếu file là tennis.txt hoặc không yêu cầu chia tập dữ liệu
        if "tennis" in file_path or not split:
            return X, y  # Chỉ trả về X_train, y_train

        # Nếu là titanic hoặc yêu cầu chia tập dữ liệu
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=520)
        print(f"Train: X {X_train.shape}, y {y_train.shape}")
        print(f"Test: X {X_test.shape}, y {y_test.shape}")
        return (X_train, y_train), (X_test, y_test)

    except Exception as e:
        print(f"Lỗi khi đọc file {file_path}: {str(e)}")
        raise

class DecisionTree:
    def __init__(self):
        self.tree = None
        self.default_label = None
        self.feature_names = None

    def fit(self, X, y):
        """
        Huấn luyện mô hình cây quyết định sử dụng thuật toán ID3
        """
        def entropy(y):
            """Tính entropy của một tập dữ liệu"""
            values, counts = np.unique(y, return_counts=True)
            probabilities = counts / len(y)
            return -np.sum(probabilities * np.log2(probabilities + 1e-10))

        def information_gain(X_data, y_data, feature_idx):
            """Tính information gain khi chia theo feature"""
            total_entropy = entropy(y_data)
            
            if isinstance(X_data, pd.DataFrame):
                feature_values = X_data.iloc[:, feature_idx]
            else:
                feature_values = X_data[:, feature_idx]
                
            values, counts = np.unique(feature_values, return_counts=True)
            weighted_entropy = 0
            
            for value, count in zip(values, counts):
                if isinstance(X_data, pd.DataFrame):
                    mask = X_data.iloc[:, feature_idx] == value
                else:
                    mask = X_data[:, feature_idx] == value
                weighted_entropy += (count / len(y_data)) * entropy(y_data[mask])
            
            return total_entropy - weighted_entropy

        def build_tree(X_data, y_data, features):
            """Xây dựng cây quyết định theo đệ quy"""
            # Nếu tất cả các mẫu đều cùng lớp, trả về nút lá
            unique_classes = np.unique(y_data)
            if len(unique_classes) == 1:
                return {'label': unique_classes[0]}
            
            # Nếu không còn feature nào, trả về nhãn phổ biến nhất
            if len(features) == 0:
                counts = np.bincount(y_data)
                return {'label': np.argmax(counts)}

            # Tính information gain cho tất cả các feature còn lại
            gains = []
            for feature in features:
                if isinstance(X_data, pd.DataFrame):
                    feature_idx = X_data.columns.get_loc(feature)
                else:
                    feature_idx = feature
                gains.append(information_gain(X_data, y_data, feature_idx))
            
            # Chọn feature có information gain cao nhất
            best_feature_idx = np.argmax(gains)
            best_feature = features[best_feature_idx]
            
            # Tạo node cho tree
            if isinstance(X_data, pd.DataFrame):
                feature_values = X_data[best_feature]
                best_feature_name = best_feature
            else:
                feature_values = X_data[:, best_feature]
                best_feature_name = best_feature
                
            # Xây dựng node mới
            node = {'feature': best_feature_name, 'children': {}}
            
            # Loại bỏ feature đã sử dụng
            remaining_features = [f for f in features if f != best_feature]
            
            # Xây dựng cây đệ quy cho mỗi giá trị của feature
            for value in np.unique(feature_values):
                if isinstance(X_data, pd.DataFrame):
                    mask = X_data[best_feature] == value
                    X_subset = X_data[mask]
                else:
                    mask = X_data[:, best_feature] == value
                    X_subset = X_data[mask]
                
                y_subset = y_data[mask]
                
                if len(y_subset) == 0:
                    # Nếu không có mẫu nào, lấy nhãn phổ biến nhất
                    counts = np.bincount(y_data)
                    node['children'][value] = {'label': np.argmax(counts)}
                else:
                    # Xây dựng cây con
                    node['children'][value] = build_tree(X_subset, y_subset, remaining_features)
            
            return node

        # Lưu lại tên feature nếu X là DataFrame
        if isinstance(X, pd.DataFrame):
            self.feature_names = X.columns.tolist()
            features = self.feature_names
        else:
            self.feature_names = [f"Feature {i}" for i in range(X.shape[1])]
            features = list(range(X.shape[1]))
        
        # Lưu lại nhãn mặc định (nhãn phổ biến nhất)
        # Đảm bảo y là array số nguyên (đã được mã hóa từ trước)
        self.default_label = np.argmax(np.bincount(y))
        
        # Xây dựng cây
        self.tree = build_tree(X, y, features)
        return self

    def predict(self, X):
        """
        Dự đoán nhãn cho dữ liệu mới sử dụng cây đã huấn luyện
        """
        def predict_sample(x, node):
            # Nếu là nút lá, trả về nhãn
            if 'label' in node:
                return node['label']
            
            # Lấy giá trị của feature
            feature = node['feature']
            if isinstance(X, pd.DataFrame):
                value = x[feature]
            elif isinstance(feature, str):
                # Nếu feature là tên và x là mảng, tìm vị trí của feature
                feature_idx = self.feature_names.index(feature)
                value = x[feature_idx]
            else:
                value = x[feature]
            
            # Kiểm tra xem giá trị có trong cây không
            if value not in node['children']:
                return self.default_label
            
            # Đệ quy theo nhánh tương ứng
            return predict_sample(x, node['children'][value])
        
        # Dự đoán cho từng mẫu
        if isinstance(X, pd.DataFrame):
            return np.array([predict_sample(X.iloc[i], self.tree) for i in range(len(X))])
        else:
            return np.array([predict_sample(x, self.tree) for x in X])

    def visualize(self):
        """
        Hiển thị cây quyết định sử dụng graphviz với kiểu dáng được cải thiện.
        
        Returns:
            Digraph: Đối tượng graphviz Digraph biểu diễn cây quyết định
        """
        def add_nodes_edges(dot, node, parent=None, edge_label=None, node_id_gen=[0]):
            node_id = str(node_id_gen[0])
            node_id_gen[0] += 1
            
            # Cải thiện kiểu dáng node
            if 'label' in node:
                label_text = f'Kết quả: {node["label"]}'
                # Sử dụng màu khác nhau cho các dự đoán khác nhau
                color = '#E74C3C' if node['label'] == 0 else '#2ECC71'  # Đỏ cho 0, Xanh lá cho 1
                dot.node(node_id, label_text, shape='box', style='filled', 
                        fillcolor=color, fontcolor='white', fontname='Arial', fontsize='12', 
                        penwidth='2')
            else:
                # Hiển thị tên thuộc tính rõ ràng hơn
                fname = node["feature"]
                dot.node(node_id, str(fname), shape='oval', style='filled', 
                        fillcolor='#3498DB', fontcolor='white', fontname='Arial', fontsize='14',
                        penwidth='2')

            # Cải thiện kiểu dáng cạnh
            if parent is not None:
                # Format nhãn rõ ràng hơn
                value_label = f'= {edge_label}'
                dot.edge(parent, node_id, label=value_label, fontname='Arial', fontsize='10',
                        penwidth='1.5', color='#555555')

            # Đệ quy để thêm các node con
            if 'children' in node:
                for val, child in node['children'].items():
                    add_nodes_edges(dot, child, node_id, edge_label=val, node_id_gen=node_id_gen)

        # Cấu hình tổng thể cho biểu đồ
        dot = Digraph()
        dot.attr(rankdir='TB', size='8,6', dpi='300', bgcolor='white')
        dot.attr('node', fontname='Arial')
        dot.attr('edge', fontname='Arial')
        
        # Thêm tiêu đề
        title = "Cây quyết định" if not self.feature_names else "Cây quyết định - Mô hình phân loại"
        dot.attr(label=title, labelloc='t', fontsize='20', fontname='Arial Bold')
        
        # Xây dựng cây
        add_nodes_edges(dot, self.tree)
        return dot

In [8]:
# Dataset 1: Tennis Dataset
print("=== Tennis Dataset ===")
tree1 = DecisionTree()
X, y = load_data("data/tennis.txt", split=False)  # Không chia tập dữ liệu test
tree1.fit(X, y)
y_hat = tree1.predict(X)
acc = accuracy_score(y, y_hat)
print(f"Tennis Dataset - Accuracy: {acc:.4f}")
tree1.visualize().render("tennis_tree", format="png", view=True)

# Dataset 2: Titanic Dataset
print("\n=== Titanic Dataset ===")
tree2 = DecisionTree()
(X_train, y_train), (X_test, y_test) = load_data("data/titanic2.txt", split=True)
tree2.fit(X_train, y_train)
y_hat_train = tree2.predict(X_train)
acc_train = accuracy_score(y_train, y_hat_train)
y_hat_test = tree2.predict(X_test)
acc_test = accuracy_score(y_test, y_hat_test)
print(f"Titanic Dataset - Train Accuracy: {acc_train:.4f}")
print(f"Titanic Dataset - Test Accuracy: {acc_test:.4f}")
tree2.visualize().render("titanic_tree", format="png", view=True)

=== Tennis Dataset ===
Đã đọc file data/tennis.txt: (14, 5) (hàng, cột)
Đã mã hóa nhãn y: [0 1]
Tennis Dataset - Accuracy: 1.0000

=== Titanic Dataset ===
Đã đọc file data/titanic2.txt: (2201, 4) (hàng, cột)
Đã mã hóa nhãn y: [0 1]
Train: X (1760, 3), y (1760,)
Test: X (441, 3), y (441,)
Titanic Dataset - Train Accuracy: 0.7915
Titanic Dataset - Test Accuracy: 0.7846


'titanic_tree.png'